In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install torch torchvision matplotlib numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 993.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs= 4
batch_size= 4
learning_rate=0.001

transform= transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset=torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset=torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,shuffle=True)

test_loader= torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=False)

clases= {'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'}

class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1= nn.Conv2d(3,6, 5)
    self.pool= nn.MaxPool2d(2,2)
    self.conv2= nn.Conv2d(6,16, 5)
    self.fc1=nn.Linear(16*5*5,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)
  def forward(self,x):
    x= self.pool(F.relu(self.conv1(x)))
    x= self.pool(F.relu(self.conv2(x)))
    x= x.view(-1,16*5*5)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x

model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

n_total_steps=len(train_loader)

for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
      images=images.to(device)
      labels=labels.to(device)
      outputs=model(images)
      loss=criterion(outputs,labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if (i+1)%2000 ==0:
        print(f'epoch {epoch+1}/{num_epochs}, Step {i+1}/{n_total_steps}, Loss: {loss.item():.4f}')
print("Finished Training")

with torch.no_grad():
  n_correct=0
  n_samples=0
  n_class_correct=[0 for i in range(10)]
  n_class_samples=[0 for i in range(10)]
  for images,labels in test_loader:
    images=images.to(device)
    labels=labels.to(device)
    outputs=model(images)
    _,predicted=torch.max(outputs,1)
    n_samples+= labels.size(0)
    n_correct+=(predicted == labels).sum().item()
    for i in range(batch_size):
      label = labels[i]
      pred=predicted[i]
      if (label== pred):
        n_class_correct[label] +=1
      n_class_samples[label] +=1
  acc= 100.0 * n_correct/n_samples
  print(f'Accuracy of the network: {acc} %')








epoch 1/4, Step 2000/12500, Loss: 2.3226
epoch 1/4, Step 4000/12500, Loss: 2.3152
epoch 1/4, Step 6000/12500, Loss: 2.2810
epoch 1/4, Step 8000/12500, Loss: 2.3040
epoch 1/4, Step 10000/12500, Loss: 2.3014
epoch 1/4, Step 12000/12500, Loss: 2.1773
epoch 2/4, Step 2000/12500, Loss: 2.5345
epoch 2/4, Step 4000/12500, Loss: 1.9673
epoch 2/4, Step 6000/12500, Loss: 1.7428
epoch 2/4, Step 8000/12500, Loss: 2.5428
epoch 2/4, Step 10000/12500, Loss: 1.1919
epoch 2/4, Step 12000/12500, Loss: 1.7501
epoch 3/4, Step 2000/12500, Loss: 1.8859
epoch 3/4, Step 4000/12500, Loss: 0.7991
epoch 3/4, Step 6000/12500, Loss: 2.0424
epoch 3/4, Step 8000/12500, Loss: 1.4460
epoch 3/4, Step 10000/12500, Loss: 3.2825
epoch 3/4, Step 12000/12500, Loss: 1.1767
epoch 4/4, Step 2000/12500, Loss: 1.2396
epoch 4/4, Step 4000/12500, Loss: 1.5281
epoch 4/4, Step 6000/12500, Loss: 1.9062
epoch 4/4, Step 8000/12500, Loss: 1.6886
epoch 4/4, Step 10000/12500, Loss: 1.4559
epoch 4/4, Step 12000/12500, Loss: 0.6378
Finished